In [6]:
import numpy as np
import random
from sklearn import preprocessing
%matplotlib inline
import matplotlib.pyplot as plt

In [69]:
# Single layer perceptron
class SLP(object):    
    def __init__(self, n_weights = 3, rate=1):
        
        # Neural network variables
        self.n_weights = n_weights # number of weights
        self.rate = rate # learning rate 
        
        # Set initial random weights for the network
        W = np.array([random.uniform(-1,1) for _ in range(self.n_weights)])
        #W = np.zeros(self.n_weights)
        W = np.insert(W, 0, 1, axis=0) # inserting bias
        self.W = W.reshape((-1,1))  
                
    
    # Activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # Activation function derivative
    def d_sigmoid(self, x): # derivate of sigmoid
        return self.sigmoid(x)*(1 - self.sigmoid(x))
    
    def V(self, X):
        return np.diag(self.d_sigmoid(X @ self.W).flatten())
    
    def H(self, X):
        #print(f'V: {self.V(X)}')
        return X.T @ self.V(X) @ X

    
    # Binary Cross-Entropy loss function
    def loss(self, Z, Y):
        return np.asscalar((-1/len(Y)) * ( Y.T @ np.log(Z + (1.e-10)) + (1 - Y).T @ np.log(1 - Z + (1.e-10)) )) 

    # Gradient of loss function
    def gradient_loss(self, Z, Y):
        dL = Y.T @ (1/Z) + (1 - Y).T @ (1/(Z - 1))
        return np.asscalar((-1/len(Y)) * dL)
    
    # Foward step
    def forward(self, X):
        V =  X @ self.W
        
        Z = self.sigmoid(V)
        
        return V,Z
    
    # Train neural network
    def train(self, X, Y):
        
        X = np.insert(X,0,1,axis=1) # inserting bias
        k = 0
        print("Epoch {}:".format(k))
        V, Z = self.forward(X)
        print("loss: {}".format(self.loss(Z,Y)))
        print("Predict: {}".format(self.predict(X,Y,bias=False)))
        loss = np.inf        
        # Perform a gradient descent algorithm
        while abs(loss) > 0.1 and k < 1000:
            k = k + 1
            #W_new = self.W - self.rate * self.gradient_loss(Z, Y) * X.T @ self.d_sigmoid(V)
            W_new = self.W + self.rate * np.linalg.pinv(self.H(X)) @ X.T @ (Y - Z)

            self.W = W_new 

            print("Epoch {}:".format(k))

            V, Z = self.forward(X)
            loss = self.loss(Z,Y)
            print(f"loss: {loss}")
            print(f"Predict: {self.predict(X,Y,bias=False)}")
    
    # Predict class of a sample
    def predict(self, X, Y, bias=True):
        if bias:
            X = np.insert(X,0,1,axis=1) # inserting bias
        Z = self.forward(X)[1].flatten().round()
        errors = 0
        for i,pred in enumerate(Z):
            if pred != Y[i]:
                errors += 1

        acc = 1 - (errors / len(Y))
        return acc       

In [70]:
nn = SLP(n_weights=30, rate=0.1)

In [71]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()

y = data.target.reshape(-1,1)

X = data.data


X = preprocessing.scale(X)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

nn.train(X_train, y_train)

Epoch 0:
loss: 3.0893689668752793
Predict: 0.18461538461538463
Epoch 1:
loss: 0.7151972266169087
Predict: 0.5824175824175823
Epoch 2:
loss: 0.5903976164593269
Predict: 0.6461538461538461
Epoch 3:
loss: 0.5074447470903242
Predict: 0.7208791208791209
Epoch 4:
loss: 0.44576768436733305
Predict: 0.7912087912087912
Epoch 5:
loss: 0.39711228150307726
Predict: 0.8461538461538461
Epoch 6:
loss: 0.35728757382767873
Predict: 0.8901098901098901
Epoch 7:
loss: 0.3238556902796361
Predict: 0.9186813186813186
Epoch 8:
loss: 0.29526484565489075
Predict: 0.9296703296703297
Epoch 9:
loss: 0.27046246728036977
Predict: 0.9406593406593406
Epoch 10:
loss: 0.2487005596350731
Predict: 0.9494505494505494
Epoch 11:
loss: 0.22942866281511193
Predict: 0.9516483516483516
Epoch 12:
loss: 0.21223080364912533
Predict: 0.9538461538461538
Epoch 13:
loss: 0.1967862597733566
Predict: 0.9582417582417583
Epoch 14:
loss: 0.18284388676063254
Predict: 0.9626373626373627
Epoch 15:
loss: 0.1702043464834851
Predict: 0.9648351648

In [73]:
nn.predict(X_test, y_test)

0.9473684210526316